In [1]:
import os
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
# 加载数据集
mnist = input_data.read_data_sets(os.path.join(os.getcwd(), "MNIST_data"), one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /Users/zoe/Documents/GitHub/July-NLP/TF/DATAGURU/Lec 07/MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /Users/zoe/Documents/GitHub/July-NLP/TF/DATAGURU/Lec 07/MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /Users/zoe/Documents/GitHub/July-NLP/TF/DATAGURU/Lec 07/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /Users/zoe/Documents/GitHub/July-NLP/TF/DATAGURU/Lec 07/MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
batch_size = 50 # 每个批次50个样本
n_batch = mnist.train.num_examples // batch_size  # 计算一共有多少个批次

In [4]:
# 输入的图片是28x28
n_inputs = 28 # 输入一行，一行有28个数据
max_time = 28 # 一共28行

lstm_size = 100 # 隐藏单元，即lstm中的block

n_classes = 10 # 10个分类

In [5]:
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

In [6]:
# 初始化权值和偏置值
weights = tf.Variable(tf.truncated_normal([lstm_size, n_classes], stddev=0.1))
biases = tf.Variable(tf.constant(0.1, shape=[n_classes]))

一般情况下，lstm的框架内已经实现了不同结构（遗忘门等）的需要的weights和biases的初始化，所以这里只要初始化一个weights和biases即可。

In [7]:
# 定义RNN网络
def RNN(X, weights, biases):
    # inputs = [batch_size, max_time, n_inputs]
    inputs = tf.reshape(x, [-1, max_time, n_inputs])
    
    # 定义LSTM基本 CELL
    lstm_cell = tf.nn.rnn_cell.LSTMCell(lstm_size)
    # final_state[0] 是cell state
    # final_state[1] 是 hidden_state
    outputs, final_state = tf.nn.dynamic_rnn(lstm_cell, inputs, dtype=tf.float32)
    results = tf.nn.softmax(tf.matmul(final_state[1], weights) + biases)
    
    return results
    

In [8]:
# 计算RNN的返回结果
prediction = RNN(x, weights, biases)

In [9]:
# 损失函数
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=prediction))

In [10]:
# 优化器
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [11]:
# 求准确率
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(prediction,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [12]:
init = tf.global_variables_initializer()

In [13]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(6):
        for batch in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step, feed_dict={x: batch_xs, y: batch_ys})
            
        acc = sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels})
        print('Iter' + str(epoch) + ', Testing Accuracy=' + str(acc))
        
        

Iter0, Testing Accuracy=0.7881
Iter1, Testing Accuracy=0.8716
Iter2, Testing Accuracy=0.9022
Iter3, Testing Accuracy=0.9126
Iter4, Testing Accuracy=0.9259
Iter5, Testing Accuracy=0.9291
